In [5]:
import torch
import torch.nn as nn
import torch.nn.functional as F

import torchvision
from torchvision import datasets, models, transforms
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler


import matplotlib.pyplot as plt 

import numpy as np

import time
import copy
import os
import random
torch.manual_seed(43)
random.seed(43)

torch.multiprocessing.set_sharing_strategy('file_system')

In [6]:
def mixup_criterion(criterion, pred, y_a, y_b, lam):
    return lam * criterion(pred, y_a) + (1 - lam) * criterion(pred, y_b)

def mixup_data(x, y, alpha=1.0, use_cuda=True):
    '''Returns mixed inputs, pairs of targets, and lambda'''
    if alpha > 0:
        lam = np.random.beta(alpha, alpha)
    else:
        lam = 1

    batch_size = x.size()[0]
    if use_cuda:
        index = torch.randperm(batch_size).cuda()
    else:
        index = torch.randperm(batch_size)
        
    mixed_x = lam * x + (1 - lam) * x[index, :]
    y_a, y_b = y, y[index]

    return mixed_x, y_a, y_b, lam

In [7]:


class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, in_planes, planes, stride=1):
        super(BasicBlock, self).__init__()
        self.conv1 = nn.Conv2d(
            in_planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3,
                               stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != self.expansion*planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, self.expansion*planes,
                          kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(self.expansion*planes)
            )

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)
        out = F.relu(out)
        return out


class Bottleneck(nn.Module):
    expansion = 4

    def __init__(self, in_planes, planes, stride=1):
        super(Bottleneck, self).__init__()
        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3,
                               stride=stride, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)
        self.conv3 = nn.Conv2d(planes, self.expansion *
                               planes, kernel_size=1, bias=False)
        self.bn3 = nn.BatchNorm2d(self.expansion*planes)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != self.expansion*planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, self.expansion*planes,
                          kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(self.expansion*planes)
            )

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = F.relu(self.bn2(self.conv2(out)))
        out = self.bn3(self.conv3(out))
        out += self.shortcut(x)
        out = F.relu(out)
        return out


class ResNet(nn.Module):
    def __init__(self, block, num_blocks, num_classes=10):
        super(ResNet, self).__init__()
        self.in_planes = 64

        self.conv1 = nn.Conv2d(3, 64, kernel_size=3,
                               stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.layer1 = self._make_layer(block, 64, num_blocks[0], stride=1)
        self.layer2 = self._make_layer(block, 128, num_blocks[1], stride=2)
        self.layer3 = self._make_layer(block, 256, num_blocks[2], stride=2)
        self.layer4 = self._make_layer(block, 512, num_blocks[3], stride=2)
        self.linear = nn.Linear(512*block.expansion, num_classes)

    def _make_layer(self, block, planes, num_blocks, stride):
        strides = [stride] + [1]*(num_blocks-1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_planes, planes, stride))
            self.in_planes = planes * block.expansion
        return nn.Sequential(*layers)

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = F.avg_pool2d(out, 4)
        out = out.view(out.size(0), -1)
        out = self.linear(out)
        return out


def ResNet18():
    return ResNet(BasicBlock, [2, 2, 2, 2])


def ResNet34():
    return ResNet(BasicBlock, [3, 4, 6, 3])


def ResNet50():
    return ResNet(Bottleneck, [3, 4, 6, 3])


def ResNet101():
    return ResNet(Bottleneck, [3, 4, 23, 3])


def ResNet152():
    return ResNet(Bottleneck, [3, 8, 36, 3])


def test():
    net = ResNet18()
    y = net(torch.randn(1, 3, 32, 32))
    print(y.size())

In [8]:
best_acc = 0  # best test accuracy
start_epoch = 0  # start from epoch 0 or last checkpoint epoch
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
    transforms.RandomErasing(p=0.5, scale=(0.02, 0.1)),
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

trainset = torchvision.datasets.CIFAR10(
    root='./data', train=True, download=True, transform=transform_train)
trainloader = torch.utils.data.DataLoader(
    trainset, batch_size=128, shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(
    root='./data', train=False, download=True, transform=transform_test)
testloader = torch.utils.data.DataLoader(
    testset, batch_size=128, shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat', 'deer',
           'dog', 'frog', 'horse', 'ship', 'truck')

net = ResNet50()
net = net.cuda()


criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.01,
                      momentum=0.9, weight_decay=5e-4)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=200)

# Training
def train(epoch):
    print('\nEpoch: %d' % epoch)
    net.train()
    train_loss = 0
    correct = 0
    total = 0
    for batch_idx, (inputs, targets) in enumerate(trainloader):
        inputs, targets = inputs.cuda(), targets.cuda()
        optimizer.zero_grad()
        
        
        use_mixup = False
        if random.random() < 0.85:
            use_mixup = True

        if use_mixup:
            data, y_a, y_b, lam = mixup_data(inputs, targets)
            outputs = net(inputs)
        else:
            outputs = net(inputs)

        _, predicted = outputs.max(1)

        if use_mixup:
            loss = mixup_criterion(criterion, outputs, y_a, y_b, lam)
        else:
            loss = criterion(outputs, targets)
        
        

        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()

        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()

    print( 'Loss: %.3f | Acc: %.3f%% (%d/%d)'
                     % (train_loss/(batch_idx+1), 100.*correct/total, correct, total))


def test(epoch):
    global best_acc
    net.eval()
    test_loss = 0
    correct = 0
    total = 0
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(testloader):
            inputs, targets = inputs.cuda(), targets.cuda()
            outputs = net(inputs)
            loss = criterion(outputs, targets)

            test_loss += loss.item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()

        print( 'Val Loss: %.3f | Acc: %.3f%% (%d/%d)'
                         % (test_loss/(batch_idx+1), 100.*correct/total, correct, total))

    # Save checkpoint.
    acc = 100.*correct/total
    if acc > best_acc:
        print('Saving..')
        state = {
            'net': net.state_dict(),
            'acc': acc,
            'epoch': epoch,
        }
        if not os.path.isdir('checkpoint'):
            os.mkdir('checkpoint')
        torch.save(state, './checkpoint/ckpt.pth')
        best_acc = acc


for epoch in range(200):
    train(epoch)
    test(epoch)
    scheduler.step()
    
print('Best val Acc: {:4f}'.format(best_acc))



Files already downloaded and verified
Files already downloaded and verified

Epoch: 0
Loss: 1.838 | Acc: 34.802% (17401/50000)
Val Loss: 1.559 | Acc: 45.780% (4578/10000)
Saving..

Epoch: 1
Loss: 1.310 | Acc: 52.934% (26467/50000)
Val Loss: 1.131 | Acc: 61.360% (6136/10000)
Saving..

Epoch: 2
Loss: 1.058 | Acc: 62.544% (31272/50000)
Val Loss: 0.925 | Acc: 67.080% (6708/10000)
Saving..

Epoch: 3
Loss: 0.868 | Acc: 69.262% (34631/50000)
Val Loss: 0.802 | Acc: 73.140% (7314/10000)
Saving..

Epoch: 4
Loss: 0.727 | Acc: 74.620% (37310/50000)
Val Loss: 0.727 | Acc: 76.010% (7601/10000)
Saving..

Epoch: 5
Loss: 0.623 | Acc: 78.336% (39168/50000)
Val Loss: 0.738 | Acc: 76.010% (7601/10000)

Epoch: 6
Loss: 0.552 | Acc: 80.922% (40461/50000)
Val Loss: 0.566 | Acc: 81.170% (8117/10000)
Saving..

Epoch: 7
Loss: 0.500 | Acc: 82.712% (41356/50000)
Val Loss: 0.633 | Acc: 79.200% (7920/10000)

Epoch: 8
Loss: 0.455 | Acc: 84.372% (42186/50000)
Val Loss: 0.500 | Acc: 83.400% (8340/10000)
Saving..

Epoch

Val Loss: 0.289 | Acc: 92.610% (9261/10000)
Saving..

Epoch: 85
Loss: 0.050 | Acc: 98.292% (49146/50000)
Val Loss: 0.270 | Acc: 92.850% (9285/10000)
Saving..

Epoch: 86
Loss: 0.050 | Acc: 98.258% (49129/50000)
Val Loss: 0.281 | Acc: 92.760% (9276/10000)

Epoch: 87
Loss: 0.049 | Acc: 98.344% (49172/50000)
Val Loss: 0.248 | Acc: 93.060% (9306/10000)
Saving..

Epoch: 88
Loss: 0.045 | Acc: 98.534% (49267/50000)
Val Loss: 0.283 | Acc: 92.810% (9281/10000)

Epoch: 89
Loss: 0.046 | Acc: 98.402% (49201/50000)
Val Loss: 0.291 | Acc: 92.240% (9224/10000)

Epoch: 90
Loss: 0.045 | Acc: 98.444% (49222/50000)
Val Loss: 0.292 | Acc: 92.690% (9269/10000)

Epoch: 91
Loss: 0.043 | Acc: 98.540% (49270/50000)
Val Loss: 0.259 | Acc: 93.110% (9311/10000)
Saving..

Epoch: 92
Loss: 0.044 | Acc: 98.508% (49254/50000)
Val Loss: 0.281 | Acc: 92.760% (9276/10000)

Epoch: 93
Loss: 0.042 | Acc: 98.600% (49300/50000)
Val Loss: 0.287 | Acc: 92.900% (9290/10000)

Epoch: 94
Loss: 0.043 | Acc: 98.530% (49265/50000)
Val 

Val Loss: 0.214 | Acc: 94.960% (9496/10000)

Epoch: 171
Loss: 0.003 | Acc: 99.916% (49958/50000)
Val Loss: 0.213 | Acc: 95.100% (9510/10000)

Epoch: 172
Loss: 0.002 | Acc: 99.934% (49967/50000)
Val Loss: 0.212 | Acc: 95.060% (9506/10000)

Epoch: 173
Loss: 0.003 | Acc: 99.926% (49963/50000)
Val Loss: 0.211 | Acc: 95.100% (9510/10000)

Epoch: 174
Loss: 0.004 | Acc: 99.906% (49953/50000)
Val Loss: 0.212 | Acc: 94.980% (9498/10000)

Epoch: 175
Loss: 0.003 | Acc: 99.926% (49963/50000)
Val Loss: 0.208 | Acc: 95.020% (9502/10000)

Epoch: 176
Loss: 0.004 | Acc: 99.908% (49954/50000)
Val Loss: 0.208 | Acc: 95.050% (9505/10000)

Epoch: 177
Loss: 0.003 | Acc: 99.930% (49965/50000)
Val Loss: 0.207 | Acc: 95.010% (9501/10000)

Epoch: 178
Loss: 0.003 | Acc: 99.930% (49965/50000)
Val Loss: 0.205 | Acc: 95.140% (9514/10000)
Saving..

Epoch: 179
Loss: 0.003 | Acc: 99.922% (49961/50000)
Val Loss: 0.205 | Acc: 95.130% (9513/10000)

Epoch: 180
Loss: 0.002 | Acc: 99.944% (49972/50000)
Val Loss: 0.205 | Acc